In [ ]:
from kde import gaussian_kde
from gpwCurlingSimulator_window import SimulationEx, Simulation, CreateShot, _GameState, _ShotVec, _ShotPos
from Config import Config
from copy import copy
import time
import numpy as np
from kr_uct import Node, KR_UCT

from functools import wraps   
%matplotlib inline
import matplotlib.figure as mfigure
import matplotlib.animation as manim
import matplotlib.pyplot as plt
from matplotlib.backends.backend_wxagg import FigureCanvasWxAgg
import wx
from wx.lib.mixins.listctrl import CheckListCtrlMixin, ListCtrlAutoWidthMixin
from copy import deepcopy
import struct




In [ ]:
from utils import *
gamestate = _GameState()
for _ in range(8):
    _, shot = CreateShot(_ShotPos(Config.TEE_X, Config.TEE_Y, 0 ))
    Simulation(gamestate, shot, Config.RAND, -1)

In [ ]:
turn = 0
st = time.time()
score_map = np.zeros((Config.IMAGE_HEIGHT, Config.IMAGE_WIDTH, 2))
for h in range(Config.IMAGE_HEIGHT):
    for w in range(Config.IMAGE_WIDTH):
        for spin in [0,1]:
            copy_state = copy(gamestate)
            _, shot = CreateShot(_ShotPos(HtoX(h), WtoY(w), spin ))
            Simulation(copy_state, shot, Config.RAND, -1)
            score_map[h][w][spin] = list(copy_state.Score)[0]
            if turn % 2 == 1:
                score_map[h][w][spin] *= -1
print(time.time() - st)

In [ ]:
from kde import gaussian_kde
from gpwCurlingSimulator_window import SimulationEx, Simulation, CreateShot, _GameState, _ShotVec, _ShotPos
from Config import Config
from copy import copy
import time
import numpy as np
from kr_uct import Node, KR_UCT

from functools import wraps   
import matplotlib.figure as mfigure
import matplotlib.animation as manim
import matplotlib.pyplot as plt
from matplotlib.backends.backend_wxagg import FigureCanvasWxAgg
import wx
from wx.lib.mixins.listctrl import CheckListCtrlMixin, ListCtrlAutoWidthMixin
from copy import deepcopy
import struct
from utils import *

    
class GamePanel(wx.Panel):
    def __init__(self, parent): 
        wx.Panel.__init__(self, parent)

        self.parent = parent
        
        self.trajectories = [[]]

        self.end = _GameState()
        self.sim_order = 'R_Y' # simulation order
        self.end.LastEnd = 9 # 10-end game
        
        self.trajectory_start_from = 120
        
        # for plot heatmap
        self.heatmap_history = []
        self.heatmap_delay = HEATMAP_DELAY
        self.kr_uct = None
        
        self.fig = mfigure.Figure(figsize=(23,10))
        self.ax1 = self.fig.add_axes([0.02,0.7,0.96,0.3])
        self.ax2 = self.fig.add_axes([0.02,0.4,0.46,0.3])
        self.ax3 = self.fig.add_axes([0.52,0.4,0.46,0.3])
        
        self.set_and_clear_stadium()

        self.canv = FigureCanvasWxAgg(self, -1, self.fig)
        self.fig.canvas.callbacks.connect('button_press_event', self.parent.parent.Get_mouseClickCoor)
        
        self.animator = manim.FuncAnimation(self.fig,self.anim, interval=1)
        
    def set_sim_order(self):            
        if not self.end.SecondTeamMove:
            self.sim_order = 'R_Y'
        else:
            self.sim_order = 'Y_R'   
    
    def set_and_clear_stadium(self):
        self.ax1.clear()
        self.ax2.clear()
        self.ax3.clear()
        self.ax3.axis('off')

        self.draw_stadium(self.ax1)
        self.draw_stadium(self.ax2)

        self.ax1.set_ylim([Config.X_PLAYAREA_MIN, Config.X_PLAYAREA_MAX])
        self.ax1.set_xlim([42.5, 0])
        self.ax2.set_ylim([Config.X_PLAYAREA_MIN, Config.X_PLAYAREA_MAX])
        self.ax2.set_xlim([Config.Y_PLAYAREA_MAX, 0])
        
        self.ax1.invert_yaxis()
        self.ax2.invert_yaxis()
        #self.ax3.invert_yaxis()
        
        self.ax1.set_aspect('equal')
        self.ax2.set_aspect('equal')
        


    def draw_stadium(self, ax):
        tee1 = (Config.TEE_Y, Config.TEE_X)
        tee2 = (42.5 - Config.TEE_Y, Config.TEE_X)
        
        house1 = plt.Circle(tee1, 1.83, color='b')
        house2 = plt.Circle(tee1, 1.22, color='w')
        house3 = plt.Circle(tee1, 0.61, color='#FF4500')
        house4 = plt.Circle(tee1, 0.15, color='w')
        house5 = plt.Circle(tee2, 1.83, color='b')
        house6 = plt.Circle(tee2, 1.22, color='w')
        house7 = plt.Circle(tee2, 0.61, color='#FF4500')
        house8 = plt.Circle(tee2, 0.15, color='w')
        ax.axhline(y=Config.TEE_X, color = 'k')
        ax.axvline(42.5 - Config.TEE_Y, color = 'k')
        ax.axvline(42.5 - Config.Y_PLAYAREA_MAX, color = 'k')
        ax.axvline(Config.TEE_Y, color = 'k')
        ax.axvline(Config.Y_PLAYAREA_MAX, color = 'k')

        ax.add_artist(house1)
        ax.add_artist(house2)
        ax.add_artist(house3)
        ax.add_artist(house4)
        ax.add_artist(house5)
        ax.add_artist(house6)
        ax.add_artist(house7)
        ax.add_artist(house8)
        
        num_shot = self.end.ShotNum
        
        if self.sim_order == 'Y_R':
            num_red_stones = int((16-num_shot)/2) + (16-num_shot) % 2
        else:
            num_red_stones = int((16-num_shot)/2)
        num_yellow_stones = (16-num_shot) - num_red_stones
        
        for i in range(num_red_stones):
            s = plt.Circle((0.145, 0.145*2*(i+1)), 0.145, edgecolor='k', facecolor='r')
            self.ax1.add_artist(s)
        for i in range(num_yellow_stones):
            s = plt.Circle((0.145 + 0.29, 0.145*2*(i+1)), 0.145, edgecolor='k', facecolor='y')
            self.ax1.add_artist(s)
    
    def draw_stone(self):      
        for obj in self.ax1.findobj(match = type(plt.Circle((1,1)))):
            if obj.get_gid() == 'rs' or obj.get_gid() == 'ys' :
                obj.remove()
                
        for obj in self.ax2.findobj(match = type(plt.Circle((1,1)))):
            if obj.get_gid() == 'rs' or obj.get_gid() == 'ys':
                obj.remove()
                
        for obj in self.ax2.findobj(match = type(plt.text(1,1,''))):
            if obj.get_gid() == 'score':
                obj.remove()
        
        for i, trajectory in enumerate(self.trajectories):
            x,y = trajectory[0]
            self.trajectories[i] = np.delete(trajectory,[0],axis = 0)
               
                
            if self.sim_order == 'R_Y':
                if i % 2==0:
                    stone_ax1 = plt.Circle((y, x), 0.145, edgecolor='k', facecolor='r', gid='rs')
                    stone_ax2 = plt.Circle((y, x), 0.145, edgecolor='k', facecolor='r', gid='rs')
                else:
                    stone_ax1 = plt.Circle((y, x), 0.145, edgecolor='k', facecolor='y', gid='ys')
                    stone_ax2 = plt.Circle((y, x), 0.145, edgecolor='k', facecolor='y', gid='ys')
            else:
                if i % 2==0:
                    stone_ax1 = plt.Circle((y, x), 0.145, edgecolor='k', facecolor='y', gid='ys')
                    stone_ax2 = plt.Circle((y, x), 0.145, edgecolor='k', facecolor='y', gid='ys')
                else:
                    stone_ax1 = plt.Circle((y, x), 0.145, edgecolor='k', facecolor='r', gid='rs')
                    stone_ax2 = plt.Circle((y, x), 0.145, edgecolor='k', facecolor='r', gid='rs')
            self.ax1.add_artist(stone_ax1)
            self.ax2.add_artist(stone_ax2)
            
            # score on ax2
            if self.end.CurEnd > 0 and self.end.ShotNum == 0:
                score = list(self.end.Score)[self.end.CurEnd-1]
            else:
                score = list(self.end.Score)[self.end.CurEnd]
            
            self.ax2.text(1, 1, 'End %i, Score[Color:%s]: %i' % (self.end.CurEnd+1,
                                                                 self.sim_order.split('_')[0],
                                                                 score),
                                                                 fontsize=15,
                                                                 horizontalalignment='right',
                                                                 verticalalignment='top',
                                                                 transform=self.ax2.transAxes,
                                                                 gid='score')
                   
    def anim(self,i):
        
        # draw heatmap
        if len(self.heatmap_history) > 0:
            self.ax3.clear()
            heatmap = self.heatmap_history.pop(0)

            self.ax3.pcolor(heatmap)
            self.ax3.set_xticks([])
            self.ax3.set_yticks([])

        # draw selected shot-simulation
        if len(self.trajectories[0]) > 0:
            self.draw_stone()

class CheckListPanel(wx.ListCtrl, CheckListCtrlMixin, ListCtrlAutoWidthMixin):
    def __init__(self, parent):
        wx.ListCtrl.__init__(self, parent, -1, style=wx.LC_REPORT | wx.SUNKEN_BORDER)
        CheckListCtrlMixin.__init__(self)
        ListCtrlAutoWidthMixin.__init__(self)                        
        self.SetSize((300,300))

class MainFrame(wx.Frame):
    def __init__(self,parent):
#         super(MainFrame,self).__init__(None, -1, size=(800, 600))

        self.heatmap_history = []

        wx.Frame.__init__(self,parent, title="Curling.", size= (1700,800))
    
        self.split_win = wx.SplitterWindow(self) # top_windwo
        self.split_win.parent = self
        self.split_win2 = wx.SplitterWindow(self.split_win) # bottom_window
        

        self.panel1 = GamePanel(self.split_win)  
        self.panel2 = wx.Panel(self.split_win2)
        self.panel3 = CheckListPanel(self.split_win2)
        self.split_win.SplitHorizontally(self.panel1, self.split_win2)
        self.split_win.SetSashGravity(0.6)
        
        self.split_win2.SplitVertically(self.panel2, self.panel3)
        self.split_win2.SetSashGravity(0.5)

        # Add Buttons to the bottom_split window and bind them to plot functions
        wx.StaticBox(self.panel2, -1, "Basic", (0, 0), size=(300, 300))
        wx.StaticText(self.panel2, -1, 'Generate Trajectory from', pos=(10, 20))
        self.Slider1 = wx.Slider(self.panel2, value = 150, minValue = 0, maxValue = 150, style = wx.SL_LABELS, pos=(30,40))  
        self.Slider1.Bind(wx.EVT_SLIDER, self.OnSliderScroll1) 
        self.Button1 = wx.Button(self.panel2, -1, "Reset", size=(80, 40), pos=(200, 20)) 
        self.Button1.Bind(wx.EVT_BUTTON, self.reset)
        wx.StaticText(self.panel2, -1, 'shot_x', pos=(10, 100))
        wx.StaticText(self.panel2, -1, 'shot_y', pos=(10, 140))
        wx.StaticText(self.panel2, -1, 'spin', pos=(10, 180))
        self.t_x = wx.TextCtrl(self.panel2, value = str(Config.TEE_X), pos = (50,100))
        self.t_y = wx.TextCtrl(self.panel2, value = str(Config.TEE_Y), pos = (50,140))
        self.t_spin = wx.TextCtrl(self.panel2, value = str(0), pos = (50,180))
        self.Button2 = wx.Button(self.panel2, -1, "Shot", size=(80, 40), pos=(200, 120)) 
        self.Button2.Bind(wx.EVT_BUTTON, self.shot)
        
        # kr_uct part
        wx.StaticBox(self.panel2, -1, "KR-UCT", (320, 0), size=(320, 300))
        wx.StaticText(self.panel2, -1, 'n_playout', pos=(330, 20))
        wx.StaticText(self.panel2, -1, 'num_samples', pos=(330, 50))
        wx.StaticText(self.panel2, -1, 'playout_depth (1 ~ 16)', pos=(330, 80))
        wx.StaticText(self.panel2, -1, 'num_initActions (>= 4)', pos=(330, 110))
        wx.StaticText(self.panel2, -1, '[KDE] band_width', pos=(330, 140))
        wx.StaticText(self.panel2, -1, 'tau', pos=(330, 170))
        wx.StaticText(self.panel2, -1, 'uct_const', pos=(330, 200))
        
        self.progress = wx.StaticText(self.panel2, -1, '0', pos=(460, 270))
        
        self.t_n_playout =  wx.TextCtrl(self.panel2, value = "400", pos = (520,20))
        self.t_num_samples = wx.TextCtrl(self.panel2, value = "100", pos = (520,50))
        self.t_playout_depth = wx.TextCtrl(self.panel2, value = "1", pos = (520,80))
        self.t_num_initActions = wx.TextCtrl(self.panel2, value = "6", pos = (520,110))
        self.t_band_width = wx.TextCtrl(self.panel2, value = "1", pos = (520,140))
        self.t_tau = wx.TextCtrl(self.panel2, value = "0.1", pos = (520,170))
        self.t_uct_const = wx.TextCtrl(self.panel2, value = "1", pos = (520,200))
        
        self.Button3 = wx.Button(self.panel2, -1, "(KR-UCT)Make Tree", size=(110, 30), pos=(340, 260)) 
        self.Button3.Bind(wx.EVT_BUTTON, self.kr_uct_maketree)
        self.Button4 = wx.Button(self.panel2, -1, "(KR-UCT)Shot", size=(80, 30), pos=(500, 260)) 
        self.Button4.Bind(wx.EVT_BUTTON, self.kr_uct_shot)
        self.Button5 = wx.Button(self.panel2, -1, "Debug", size=(40, 30), pos=(590, 260)) 
        self.Button5.Bind(wx.EVT_BUTTON, self.debug)
        self.Button6 = wx.Button(self.panel2, -1, "Select All", size=(90, -1), pos=(740, 20))
        self.Button6.Bind(wx.EVT_BUTTON, self.OnSelectAll)
        self.Button7 = wx.Button(self.panel2, -1, "Deselect All", size=(90, -1), pos=(740, 60)) 
        self.Button7.Bind(wx.EVT_BUTTON, self.OnDeselectAll)
        #  panel 3 (table)
        self.panel3.OnCheckItem = self.OnTable # to bind it
        self.panel3.InsertColumn(0, 'Action (Power, Angle, spin)', width=170)
        self.panel3.InsertColumn(1, 'Q_value (Score Difference)', width=170)
        self.panel3.InsertColumn(2, 'n_visits')
        self.panel3.InsertColumn(3, 'E[v|a]')
        self.panel3.InsertColumn(4, 'W(a)')
        self.panel3.InsertColumn(5, 'Lower Confidence Bound')
        
        # Demo state part
        wx.StaticBox(self.panel2, -1, "Examples", (660, 0), size=(70, 300))
        self.Button10 = wx.Button(self.panel2, -1, "state 0", size=(50, 30), pos=(670, 20))
        self.Button10.Bind(wx.EVT_BUTTON, self.state_0) 
        
        self.Button11 = wx.Button(self.panel2, -1, "state 1", size=(50, 30), pos=(670, 70))
        self.Button11.Bind(wx.EVT_BUTTON, self.state_1) 
      
    def Get_mouseClickCoor(self, event):
        y, x = event.inaxes.transData.inverted().transform((event.x, event.y))
        self.t_x.SetLabel(str(x))
        self.t_y.SetLabel(str(y))
    
    def OnSelectAll(self, event):
        if self.panel1.kr_uct == None:
            wx.MessageBox('Please make the tree first!', 'Warning', wx.OK | wx.ICON_INFORMATION)
            return
        
        num = self.panel3.GetItemCount()
        for i in range(num):
            self.panel3.CheckItem(i)

    def OnDeselectAll(self, event):
        if self.panel1.kr_uct == None:
            wx.MessageBox('Please make the tree first!', 'Warning', wx.OK | wx.ICON_INFORMATION)
            return
        
        num = self.panel3.GetItemCount()
        for i in range(num):
            self.panel3.CheckItem(i, False)

    def set_state(self, arr, traj):
        self.heatmap_history = []
        self.panel1.heatmap_history = []
        self.panel1.ax3.clear()
        self.panel1.ax3.axis('off')
     
        
        for i in range(len(arr)):
            _, ShotVec = CreateShot(_ShotPos(arr[i,0], arr[i,1], arr[i,2]))
            success, ResShot, tra = SimulationEx(self.panel1.end, ShotVec, 0., 0.,-1)
            
        trajectories = []
        last_positions = self.panel1.end.body
        for n in range(self.panel1.end.ShotNum):
            last_position = last_positions[n]
            trajectories.append([last_position])        
    

        self.panel1.trajectories = trajectories
            
    def state_0(self, event):
        self.reset()
        test_state = np.array([[2.67, 4.7,0],
                               [2.24, 5.21,0],
                               [2.88, 6.5,0],
                               [1.73, 8.61,0],
                               [2.32, 6.68,0]])
        self.set_state(test_state,False)
        
    def state_1(self, event):
        self.reset()
#         test_state = np.array([[2.42846719002, 4.9692406401,0],
#                                 [3.20374144525, 6.09205163043,0],
#                                 [2.34826640499, 7.963403281,0],
#                                 [1.94726247987, 5.07617502013,0],
#                                 [1.97399607488, 7.99013687601,0],
#                                 [3.17700785024, 4.9692406401,0]]) 
        test_state = np.array([[ -0.11598411,  10.42058945,   0.        ],
                            [ 1.77547181,  7.04998922,  0.        ],
                            [ 2.20567393,  7.39408875,  0.        ],
                            [ 2.08004832,  3.58526206,  1.        ],
                            [ 1.70991755,  5.31220627,  1.        ],
                            [ 1.44734478,  0.23787722,  1.        ]]) 
        self.set_state(test_state,False)    

    def OnTable(self, index, flag): 
        
        for obj in self.panel1.ax1.findobj(match = type(plt.Circle((1,1)))):
            if obj.get_gid() == 'expected_stone':
                obj.remove()
        for obj in self.panel1.ax2.findobj(match = type(plt.Circle((1,1)))):
            if obj.get_gid() == 'expected_stone':
                obj.remove()
        
        
        table_num_row = self.panel3.GetItemCount()
        virtual_X = None
        virtual_Y = None
        self.t_x.SetLabel(str(virtual_X))
        self.t_y.SetLabel(str(virtual_Y))
        for i in range(table_num_row):
            if self.panel3.IsChecked(i):
                virtual_X = float(self.panel3.GetItemText(i).split(',')[0][1:])
                virtual_Y = float(self.panel3.GetItemText(i).split(',')[1])
                virtual_SPIN = int(self.panel3.GetItemText(i).split(',')[2][:-1])
                
                # green -> clock wise
                # blue -> counter clock wise
                if virtual_SPIN == 0: 
                    ax1_s = plt.Circle((virtual_Y, virtual_X), 0.145, alpha=0.75,edgecolor='k', facecolor='g', gid='expected_stone')
                    ax2_s = plt.Circle((virtual_Y, virtual_X), 0.145, alpha=0.75,edgecolor='k', facecolor='g', gid='expected_stone')
                else:
                    ax1_s = plt.Circle((virtual_Y, virtual_X), 0.145, alpha=0.75,edgecolor='k', facecolor='b', gid='expected_stone')
                    ax2_s = plt.Circle((virtual_Y, virtual_X), 0.145, alpha=0.75,edgecolor='k', facecolor='b', gid='expected_stone')
                self.panel1.ax1.add_artist(ax1_s)
                self.panel1.ax2.add_artist(ax2_s)

        self.t_x.SetLabel(str(virtual_X))
        self.t_y.SetLabel(str(virtual_Y))
    
    def OnSliderScroll1(self, event): 
        obj = event.GetEventObject() 
        self.panel1.trajectory_start_from = obj.GetValue() 

    def reset(self, event=None):
        self.panel1.trajectories = [[]]
        self.panel1.end = _GameState()
        self.panel1.end.LastEnd = 10
        self.panel1.heatmap_history = []
        self.panel1.kr_uct = None
        self.panel1.set_and_clear_stadium()
        table_num_row = self.panel3.GetItemCount()
        for i in range(table_num_row):
            self.panel3.DeleteItem(0)

    def debug(self, event):
        if self.panel1.kr_uct == None:
            wx.MessageBox('Please make the tree first!', 'Warning', wx.OK | wx.ICON_INFORMATION)
            return 
        
        # draw heatmap
        self.panel1.heatmap_history = deepcopy(self.heatmap_history)
        
            
    def simulate_shot(self, x, y, spin):
        # if it is first shot in this end, ordering the color
        if self.panel1.end.ShotNum == 0:
            self.panel1.set_sim_order()
        
        _, ShotVec = CreateShot(_ShotPos(x, y, spin))
        success, ResShot, tra = SimulationEx(self.panel1.end, ShotVec,
                                             Config.RAND / 2, Config.RAND * 2,-1)        
        
        last_positions = self.panel1.end.body
        self.panel1.set_and_clear_stadium()
                          
        trajectories = []
        
        
        if self.panel1.end.ShotNum == 0:
            num_stones = 16
        else:
            num_stones = self.panel1.end.ShotNum
        
        for n in range(num_stones):
            last_position = last_positions[n]
            trajectory = tra[self.panel1.trajectory_start_from:,n,:]
            idx_stop = np.where(trajectory==last_position)[0][-1]
            trajectory[idx_stop+1:]=last_position
            trajectories.append(trajectory)
                          
        self.panel1.trajectories = trajectories
            
        # forget prev tree
        self.panel1.kr_uct = None
        
    def shot(self,event):
        # remove kr-uct debug info
        self.heatmap_history = []
        self.panel1.heatmap_history = []

        table_num_row = self.panel3.GetItemCount()
        for i in range(table_num_row):
            self.panel3.DeleteItem(0)
            
        self.simulate_shot(float(self.t_x.GetValue()), float(self.t_y.GetValue()), int(self.t_spin.GetValue()))
                    

                
    def kr_uct_shot(self,event):
        # remove kr-uct debug info
        self.heatmap_history = []
        self.panel1.heatmap_history = []
        table_num_row = self.panel3.GetItemCount()
        for i in range(table_num_row):
            self.panel3.DeleteItem(0)
        
        if self.panel1.kr_uct == None:
            wx.MessageBox('Please make the tree first!', 'Warning', wx.OK | wx.ICON_INFORMATION)
            return
            
        _, best_action, best_spin = self.panel1.kr_uct.lcb_select(self.panel1.kr_uct.root)
        self.simulate_shot(best_action[0], best_action[1], spin=best_spin)     
        
    def kr_uct_maketree(self, event):
        heatmap_history = []

#         self.panel1.kr_uct = KR_UCT(int(self.t_num_initActions.GetValue()),
#                                     x_range = X_RANGE,
#                                     y_range = Y_RANGE,
#                                     kde = GAUSSIAN_KDE,
#                                     band_width = float(self.t_band_width.GetValue()),
#                                     cov_mat = COV_MAT,
#                                     exec_var = EXEC_VAR,
#                                     tau = float(self.t_tau.GetValue()),
#                                     num_samples=int(self.t_num_samples.GetValue()),
#                                     playout_depth=int(self.t_playout_depth.GetValue()),
#                                     n_playout=int(self.t_n_playout.GetValue()),
#                                     uct_const=float(self.t_uct_const.GetValue()),
#                                     policy_net=POLICY_NET)
        self.panel1.kr_uct = KR_UCT(
                num_initActions = 10,
                x_range = X_RANGE,
                y_range = Y_RANGE,
                kde = GAUSSIAN_KDE,
                band_width = 1,
                cov_mat = COV_MAT,
                exec_var = EXEC_VAR,
                tau = 0.02,
                num_samples=100,
                playout_depth=1,
                n_playout=100,
                uct_const=0.3,
                policy_net = POLICY_NET)
        
        if self.panel1.end.SecondTeamMove:
            root_color = 'Y'
        else:
            root_color = 'R'
        
        self.panel1.kr_uct.root = Node(None, root_color)
        for n in range(int(self.t_n_playout.GetValue())):
            self.progress.SetLabel(str(n+1))
            vstate = copy(self.panel1.end)
            self.panel1.kr_uct.playout(vstate)
                

        # clear the table
        table_num_row = self.panel3.GetItemCount()
        for i in range(table_num_row):
            self.panel3.DeleteItem(0)
        # write the table
        node = self.panel1.kr_uct.root
        
        for spin in [0,1]:
            A = np.array([i for i in node.children[spin].keys()]) # action sets
            n_A = np.array([i.n_visits for i in node.children[spin].values()]) # n_visits of action set
            v_A = np.array([i.Q for i in node.children[spin].values()]) # empirical value of action set

            # nominator of E[v|a]
            self.panel1.kr_uct.kde.set_dataset(A.T)
            self.panel1.kr_uct.kde.set_weights(n_A * v_A)
            nomi_E_v_a = self.panel1.kr_uct.kde.evaluate(A.T)

            # W(a) = denominator of E[v|a]
            self.panel1.kr_uct.kde.set_dataset(A.T)
            self.panel1.kr_uct.kde.set_weights(n_A)
            denomi_E_v_a = self.panel1.kr_uct.kde.evaluate(A.T)

            E_v_a = nomi_E_v_a / denomi_E_v_a

            bound = np.array([np.sqrt(np.log(denomi_E_v_a.sum())/ W_a) for W_a in denomi_E_v_a])

            LCB_array = E_v_a - self.panel1.kr_uct.uct_const * bound
            
            if spin == 0:
                A_all = A
                n_A_all = n_A
                v_A_all = v_A
                E_v_a_all = E_v_a
                denomi_E_v_a_all = denomi_E_v_a
                LCB_array_all = LCB_array
                spin_all = [spin] * len(LCB_array)
            else:
                A_all = np.concatenate((A_all, A))
                n_A_all = np.concatenate((n_A_all, n_A))
                v_A_all = np.concatenate((v_A_all, v_A))
                E_v_a_all = np.concatenate((E_v_a_all, E_v_a))
                denomi_E_v_a_all = np.concatenate((denomi_E_v_a_all, denomi_E_v_a))
                LCB_array_all = np.concatenate((LCB_array_all, LCB_array))
                spin_all = np.concatenate((spin_all, [spin] * len(LCB_array)))
      
        for i in sorted(range(len(LCB_array_all)), reverse = True,key = lambda k: LCB_array_all[k]):
            index = self.panel3.InsertStringItem(PLATFORM_C_MAXINT, '(%.5f, %.5f, %d)' % (A_all[i,0], A_all[i,1], spin_all[i])) # add action
            self.panel3.SetStringItem(index, 1, '%.2f' % v_A_all[i])
            self.panel3.SetStringItem(index, 2, '%i' % n_A_all[i])
            self.panel3.SetStringItem(index, 3, '%.2f' % E_v_a_all[i])
            self.panel3.SetStringItem(index, 4, '%.2f' % denomi_E_v_a_all[i])
            self.panel3.SetStringItem(index, 5, '%.2f' % LCB_array_all[i])
    
        debug = np.zeros((30,30))
        print([list(i) for i in list(self.panel1.end.body)])
        for idx in range(self.panel1.end.ShotNum):
            x,y = self.panel1.end.body[idx][0], self.panel1.end.body[idx][1]
            h, w = XtoH(x), YtoW(y)
            if idx % 2 ==0:
                debug[h][w] = 1
            else:
                debug[h][w] = -1
        self.panel1.heatmap_history = [np.rot90(np.rot90(debug))]                            
        """
        positions = []
        
        for x in np.linspace(X_RANGE[0], X_RANGE[1] , X_BIN_SIZE):
            for y in np.linspace(Y_RANGE[1], Y_RANGE[0] , Y_BIN_SIZE):
                positions.append([x,y])
        positions = np.array(positions).T

        # nominator of E[v|a]
        kernel_1 = self.panel1.kr_uct.kde(A.T,
                                          cov_mat = self.panel1.kr_uct.cov_mat,
                                          weights = n_A * v_A)
        nomi_E_v_a = kernel_1.evaluate(positions)

        # W(a) = denominator of E[v|a]
        kernel_2 = self.panel1.kr_uct.kde(A.T,
                                          cov_mat = self.panel1.kr_uct.cov_mat,
                                          weights = n_A)
        denomi_E_v_a = kernel_2.evaluate(positions)
        
        E_v_a = nomi_E_v_a / denomi_E_v_a
        
        #self.panel1.heatmap_history = [ denomi_E_v_a.reshape((Y_BIN_SIZE, X_BIN_SIZE))]
        self.panel1.heatmap_history = [E_v_a.reshape((Y_BIN_SIZE, X_BIN_SIZE))]
        """
            
            

        
        

              
"""
    STATIC VALUE
"""
# KR-UCT param

from PolicyNet import PolicyNet
Config.RAND = 0.145

Config.NUM_CONV_LAYERS=7
Config.DEVICE = 'gpu:0'
POLICY_NET = PolicyNet(Config.DEVICE, Config.NETWORK_NAME, train_models = False)

POLICY_NET.load('pretrain_checkpoint/both_uct/')
# PRETRAIN_POLICY_NETWORK_PATH = 'pretrain_checkpoint/both_spin_layer7/'
# POLICY_NET.load(PRETRAIN_POLICY_NETWORK_PATH)
POLICY_NET = None

GAUSSIAN_KDE = gaussian_kde
X_RANGE = [Config.X_PLAYAREA_MIN, Config.X_PLAYAREA_MAX]
Y_RANGE = [0, Config.Y_PLAYAREA_MAX]
#EXEC_VAR = np.diag([0.2, 0.5])
#COV_MAT = np.array([[0.0625, 0], [0., 0.25]])
COV_MAT = np.array([[(0.145 / 2) ** 2, 0.], [0., (0.145 * 6) ** 2]])
#COV_MAT = np.array([[(0.145 * 4) ** 2, 0.], [0., (0.145 * 4) ** 2]])
EXEC_VAR = COV_MAT

# Debug (Draw heatmap)
X_BIN_SIZE = 100
Y_BIN_SIZE = 100
LABEL_SPACING = 20
#HEATMAP_DELAY = 20


LABEL_SPACING = 20
HEATMAP_DELAY = 20


# To make CheakList Panel
PLATFORM_C_MAXINT = 2 ** (struct.Struct('i').size * 8 - 1) - 1
 
print('Currently, only 1 end game is possible')

app = wx.App()
w = MainFrame(None)
w.Show(True)
app.MainLoop()
